# L7a: Estimating Model Parameters
In this lecture, we'll look at some approaches to estimate unknown model parameters from data. The key topics we'll cover are:

* __Least squares minimization__ is an optimization problem used to estimate model parameters by minimizing the squared difference between experimental data and model predictions. This problem can be solved using a variety of optimization algorithms.
* __Maximum likelihood estimation__ is a statistical method used to estimate model parameters by maximizing the likelihood of observing the experimental data given the model. This method is particularly useful when the data is noisy and the model is probabilistic.
* __Gradient-based optimization__ is a general approach to solving optimization problems by iteratively updating the model parameters in the direction of the gradient of the objective function. This method is widely used in machine learning and scientific computing. 
* __Heuristic optimization__ is a class of optimization algorithms that do not rely on the gradient of the objective function. These algorithms are often used when the derivatives of the objective function are challenging to compute or the objective function is non-convex, i.e., many local minima exist.

## Problem
Suppose we have an oridinary differential equation model, e.g., a system of mRNA and protein balances, of the form:
$$
\begin{align*}
\dot{x} & = f(x, u, \theta)\\
y & = g(x, \theta)\\
x(0) & = x_0
\end{align*}
$$
where $x \in \mathbb{R}^n$ is the state vector (e.g., mRNA and protein measurements), $u \in \mathbb{R}^m$ is the input vector, $\theta \in \mathbb{R}^p$ is the (unkown) parameter vector, $y \in \mathbb{R}^q$ is the output vector, and $f$ and $g$ are the state and output functions, respectively. 
* __Challenge__: Given a set of inputs $\left\{u^{(t)}_i \mid i = 1, \dots, m\right\}$ and output measurements $\left\{y^{(t)}_i \mid i = 1, \dots, q\right\}$ at time $t$, we need to estimate the unknown parameters $\theta\in\mathbb{R}^{p}$ that _best_ describes the data.

The different ways to solve this problem come down to (for the most part) how we define the term _best_. However, before we get into the details, let's think about why this problem is hard to solve:

* __Non-Identifiability and Sloppiness__: Many systems biology models exhibit non-identifiability, meaning multiple sets of parameter values can fit the data equally well. Additionally, models often display "sloppiness," where parameters have sensitivities spread over many orders of magnitude, making precise estimation difficult.
* __High-Dimensional Parameter Spaces__: Biological models can have _many_ unknown parameters, leading to high-dimensional search spaces. This complexity increases computational demands and makes optimization challenging.
* __Noisy and Limited Data__: Experimental data used for parameter estimation in systems biology (and to a lesser extent in some synthetic applications) are often noisy and limited in scope, which complicates the optimization process by introducing additional local minima in the objective function.
* __Overfitting__: Models with many parameters can overfit the data, capturing random fluctuations rather than underlying trends, which results in poor predictive performance on new data.

## Least Squares Minimization
One common approach to estimating model parameters is to minimize the sum of squared differences between the experimental data and model predictions. This method is known as least squares minimization and is widely used in regression analysis and machine learning. Suppose we run an experiment and collect data $\left\{y^{(t)}_i \mid i = 1, \dots, q\right\}$ at time $t=1,2,\dots,T$ for some input. The objective function to minimize is given by:
$$
\begin{align*}
\text{minimize} & \quad J(\theta) = \frac{1}{2}\sum_{t=1}^{T}\sum_{i=1}^{q} w_{i,t}\cdot\left(y^{(t)}_i - g_{i}(x^{(t)}, \theta)\right)^2
\end{align*}
$$
where $J(\theta)$ is the objective function, $y^{(t)}_i$ are the experimental measurement $i$ at time $t$, $g(x^{(t)}_i, \theta)$ are the model predictions, 
$w_{i,t}$ are weight coefficients, and $\theta$ are the unknown parameters to estimate. The goal is to find the parameter values $\theta$ that minimize the sum of squared differences between the data and model predictions.
* To solve this problem, we need to solve the model equations for each time step $t$ to obtain the state vector $x^{(t)}$ and then compute the output $g(x^{(t)}, \theta)$. We can then use an _optimization algorithm_ to minimize the objective function $J(\theta)$ with respect to the parameters $\theta$.
* The choice of the weight coefficients $w_{i,t}$ is crucial and depends on the nature of the data. For example, if the data is noisy, we might assign higher weights to data points with lower variance to reduce the impact of outliers on the optimization process.

## Maximum Likelihood Estimation (MLE)
Another common approach to estimating model parameters is maximum likelihood estimation (MLE). This method is particularly useful when the data is noisy. Suppose we have a probabilistic model that describes the likelihood of observing the experimental data given the model and parameters. The likelihood function is given by:
$$
\begin{equation*}
L_{t}(y^{(t)},\,g(x^{(t)}, \theta)) = \prod_{i=1}^{q}P(y_{i}^{(t)}\mid\,g_{i}^{(t)}(x^{(t)}, \theta))
\end{equation*}
$$
where $P(y_{i}^{(t)}\mid\,g_{i}(x^{(t)}, \theta))$ is the probability of observing experimental output $y_{i}^{(t)}$ given the model predictions $g_{i}(x^{(t)}, \theta)$, which are functions of the parameters, at time $t$. The goal of MLE is to find the parameter values $\theta$ that _maximizes_ the likelihood of observing the experimental data given the model.

### Log-Likelihood Function
It's hard to maximize the likelihood function directly (because of the product), so we take the logarithm of the likelihood function to simplify the optimization:
$$
\begin{align*}
\log{L}(\theta) & = \sum_{t=1}^{T}\sum_{i=1}^{q}\log{P(y_{i}^{(t)}\mid\,g_{i}(x^{(t)}, \theta))}\\
\end{align*}
$$
Different probability functions and assumptions made about the noise in the system can lead to different likelihood functions.
For example, if the data is normally distributed, we might use a Gaussian likelihood function of the form:
$$
\begin{align*}
P(y_{i}^{(t)}\mid\,g_{i}(x^{(t)}, \theta)) & = \frac{1}{\sqrt{2\pi\sigma_{i}^{2}}}\exp\left(-\frac{(y_{i}^{(t)} - g_{i}(x^{(t)}, \theta))^{2}}{2\sigma_{i}^{2}}\right)
\end{align*}
$$
where $\sigma_{i}^{2}$ is the variance of the noise in the system. The goal is to find the parameter values $\theta$ that maximize the log-likelihood function.

## Gradient descent
Gradient descent is a numerical search algorithm that minimizes a function by iteratively adjusting the parameters in the opposite direction of the gradient. Suppose there exists an objective function, e.g., the negative log-likelihood $\mathcal{L}(\theta)$ that we want to minimize with respect to parameters $\theta\in\mathbb{R}^{p}$. We assume $\mathcal{L}(\theta)$ is _at least once differentiable_ with respect to the parameters, i.e., we can compute the gradient $\nabla_{\theta}{\mathcal{L}}(\theta)$. The gradient points in the direction of the steepest increase of the function. Thus, we can iteratively update the parameters to minimize the objective function using the update rule:
$$
\begin{equation*}
\theta_{k+1} = \theta_{k} - \alpha(k)\cdot\nabla_{\theta}\mathcal{L}(\theta_{k})\quad\text{where}{~k = 0,1,2,\dots}
\end{equation*}
$$
where $k$ denotes the iteration index, and $\nabla_{\theta}\mathcal{L}(\theta)$ is the gradient of the negative log-likelihood function with respect to the parameters $\theta$.
* __What is $\alpha(k)$?__ The (hyper) parameter $\alpha(k)>0$ is the _learning rate_ which can be a function of the iteration count $k$. This is a user-adjustable parameter, and we'll assume it's constant for today.
* __Stopping?__ Gradient descent will continue to iterate until a stopping criterion is met, i.e., $\lVert\theta_{k+1} - \theta_{k}\rVert\leq\epsilon$ or the maximum number of iterations is reached, or some other stopping criterion is met, i.e., the gradient is small at the current iteration $\lVert\nabla_{\theta}\mathcal{L}(\theta_{k})\rVert\leq\epsilon$.

Pusedocode for a naive gradient descent algorithm (for a fixed learning rate) is shown in the [CHEME 5820 lecture notes](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-3/L3c/docs/Notes.pdf).

## Alternatives to gradient descent
Gradient descent is a powerful optimization algorithm, but it has some limitations. For example, it can be slow to converge, especially when the objective function is non-convex or has a complex landscape. Additionally, gradient descent requires the gradient of the objective function, which can be computationally expensive to compute for large models. In these cases, alternative optimization algorithms may be more suitable. Some common alternatives to gradient descent include:

* __Genetic Algorithms__: These are population-based optimization methods inspired by natural selection and genetics. They use processes like mutation, crossover, and selection to evolve a population of candidate solutions towards optimal values. Genetic algorithms are particularly useful for navigating complex search spaces without requiring gradient information, making them suitable for black-box optimization problems134.
* __Simulated Annealing__: This algorithm mimics the annealing process in metallurgy, where the temperature is gradually lowered to achieve a stable crystal structure. It starts with a high "temperature" that allows for extensive exploration of the solution space and gradually cools down, reducing the likelihood of accepting worse solutions. This method is effective for avoiding local minima and can be applied to problems where gradient information is not available134.
* __Particle Swarm Optimization (PSO)__: PSO is another population-based method that involves a swarm of particles moving through the search space. Each particle adjusts its position based on its own experience and the experience of its neighbors, leading to a collective convergence towards optimal solutions. PSO is known for its ability to handle complex optimization problems and can outperform gradient descent in certain scenarios.

## Today?
That's a wrap! What are some of the interesting things we discussed today?